# Criminality Across Vancouver Neighbourhoods

By: Isabela Lucas Bruxellas, Sam Thorne, Sean Meng

## Introduction

#### Background Information

Despite nationwide crime rates declining since the 1990s in Canada (STATCAN 2021), many Canadians still fear crime, especially in their local communities (JUSTICE CANADA, 2022). Current research suggests that effective crime control strategies embrace the importance of regular citizens in preventing crime and enhance their ability to do so (USCIDA et al., 2014). This research focuses on empowering citizens by providing an understanding of overall and crime-specific proportions across the communities of Vancouver. This analysis will infer which districts in Vancouver have proportionally higher and lower crime rates.



#### Research Question

Is there a difference between the average annual proportion of crimes in Vancouver across different districts? 

To answer this question, we will explore the proportion of crimes grouped per year from **2003** to **2021** and across the **Central**, **East**, **South** and **West** districts of Vancouver. We will calculate the mean proportion of crime that lies in each district annually and find the interquartile range. We will also analyze trends for different crime types to explore significant pattern changes.

#### The Dataset, Trustworthiness

The dataset is provided by the Vancouver Police Department (VDC) and was collected from the PRIME BC Police Records Management System (RMS)on October 21st, 2022. As a result, it is trustworthy and unbiased because it contains all police-reported crimes in Vancouver. It includes crime **type**, **time**, and **location**.

## Methods and Results

In [8]:
set.seed(101)

library(tidyverse)
library(repr)
library(digest)
library(infer)
library(gridExtra)
library(cowplot)
library(RColorBrewer)

ERROR: Error in library(tidyverse): there is no package called ‘tidyverse’


#### Data import and cleaning

In [9]:
# Data uploaded to open GitHub repository
# Data reading 
crimedata <- read_csv('https://raw.githubusercontent.com/samnthorne/STAT201_Group40/main/crimedata_csv_AllNeighbourhoods_AllYears.csv')
head(crimedata) 

ERROR: Error in read_csv("https://raw.githubusercontent.com/samnthorne/STAT201_Group40/main/crimedata_csv_AllNeighbourhoods_AllYears.csv"): could not find function "read_csv"


We keep the `TYPE`, `YEAR` and `NEIGHBOURHOOD` columns and remove rows containing NA value. We define the districts via the `NEIGHBOURHOOD` column with reference to Globe Media Ltd. (2022). 

In [10]:
# Dropping unnecessary columns and filtering out NA values.
selected_crimedata <- crimedata %>%
    select('YEAR', 'NEIGHBOURHOOD', 'TYPE') %>%
    filter(!is.na(NEIGHBOURHOOD)) %>%
    filter(!is.na(YEAR)) # drop NA values in the neighbourhood column and in the year column. 

# defining the districts of Vancouver. 
central = c('Central Business District', 'Stanley Park', 'West End')
west = c('Fairview', 'Kitsilano', 'West Point Grey',  
        'Shaughnessy', 'Musqueam', 'Arbutus Ridge', 'South Cambie', 'Oakridge')
east = c('Grandview-Woodland', 'Mount Pleasant', 'Kensington-Cedar Cottage',
         'Riley Park', 'Strathcona', 'Hastings-Sunrise', 'Killarney')
south = c('Dunbar-Southlands', 'Kerrisdale', 'Marpole','Sunset', 'Victoria-Fraserview', 
          'Renfrew-Collingwood','Killarney')

# Creating a new column in the data frame of district
# value can be Central, West, East, or South depending on neighbourhood
# Can drop the neighbourhood column here too as from here onwards we will exclusively look at district.
district_crimedata <- selected_crimedata %>%
    mutate(DISTRICT = case_when(
        NEIGHBOURHOOD == 'Central Business District' | NEIGHBOURHOOD == 'Stanley Park' | NEIGHBOURHOOD == 'West End' ~ 'Central',
        NEIGHBOURHOOD == 'Fairview' | NEIGHBOURHOOD == 'Kitsilano'| NEIGHBOURHOOD =='West Point Grey'|
        NEIGHBOURHOOD == 'Shaughnessy'| NEIGHBOURHOOD == 'Musqueam'|
        NEIGHBOURHOOD == 'Arbutus Ridge'| NEIGHBOURHOOD == 'South Cambie' | NEIGHBOURHOOD == 'Oakridge' ~ 'West',
        NEIGHBOURHOOD == 'Grandview-Woodland'| NEIGHBOURHOOD == 'Mount Pleasant'| 
        NEIGHBOURHOOD == 'Kensington-Cedar Cottage'| NEIGHBOURHOOD == 'Riley Park'| NEIGHBOURHOOD == 'Strathcona'|
        NEIGHBOURHOOD == 'Hastings-Sunrise' ~ 'East',
        NEIGHBOURHOOD == 'Dunbar-Southlands'|NEIGHBOURHOOD == 'Kerrisdale' | NEIGHBOURHOOD == 'Marpole'|NEIGHBOURHOOD == 'Sunset'|
        NEIGHBOURHOOD == 'Victoria-Fraserview'|NEIGHBOURHOOD == 'Renfrew-Collingwood'|NEIGHBOURHOOD == 'Killarney'~'South')) %>%
    select(-NEIGHBOURHOOD)
    # filter(is.na(DISTRICT)) # use to make sure every neighbourhood in the data has been included.

# the minimum year in this data is 2003 so this is how far back we will be looking.
print(paste("The first year to be included in our analysis is", min(crimedata$YEAR)))
# 2021 will be the maximum year in our data as 2022 has not finished yet so it's numbers are not
# representative of an entire year of crime.

# removing the year 2022 from the data
year_crime_data <- district_crimedata %>%
    filter(YEAR != 2022)

# Next we grouped the type of crime into more broad categories than what is in the original data set.
# We also removed car accident type to reduce noise. We determined it was unnnecessary when trying to 
# determine the safest district in regards to crime.
crime_data <- year_crime_data %>%
    mutate(TYPE = case_when(
        TYPE == "Break and Enter Commercial"|TYPE == "Break and Enter Residential/Other" ~ "Break and Enter",
        TYPE == "Other Theft"|TYPE == "Theft from Vehicle"|TYPE == "Theft of Bicycle"|TYPE == "Theft of Vehicle" ~ "Theft",
        TYPE == 'Homicide' ~ 'Homicide', TYPE == 'Mischief'~'Mischief', TYPE == 'Offence Against a Person'~'Offence Against a Person')) %>%
    filter(!is.na(TYPE))

print("This is a piece of the wrangled data frame")
head(crime_data)

ERROR: Error in crimedata %>% select("YEAR", "NEIGHBOURHOOD", "TYPE") %>% filter(!is.na(NEIGHBOURHOOD)) %>% : could not find function "%>%"


#### Data Plots

In [11]:
options(repr.plot.width = 18, repr.plot.height = 8)

years_vis <- crime_data %>%
    ggplot(aes(x = YEAR, fill = TYPE))+
    geom_histogram(binwidth = 1, color = 'white')+
    theme_classic() +
    theme(legend.position="right", 
          text = element_text(size = 15)) +
    labs(title = "Frequency of crimes and type of crime in each district in Vancouver \n through the years 2003 to 2021",
         x = "Year",
         y = "Number of Crimes",
         fill = "Type of Crime")+
    facet_wrap(~DISTRICT, ncol = 4)+
    scale_fill_brewer(palette = 'Set2') # colorblind friendly palette.
years_vis

ERROR: Error in crime_data %>% ggplot(aes(x = YEAR, fill = TYPE)): could not find function "%>%"


The plots above show the amount of crime in all four districts from 2003 to 2021. The stacks in the bar graph  colour-code the types of crimes for further analysis.

#### Estimates

We should calculate the proportion of crime in each district per year, then calculate the average proportion of crime in each district from 2003 to 2021.

In [8]:
# Added another column to the data which holds the proportion of crime for the corresponding district and year
# of the row.
prop_data <- crime_data %>%
    group_by(YEAR) %>%
    mutate(num_year = n()) %>%
    ungroup() %>%
    group_by(YEAR, DISTRICT) %>%
    mutate(PROP = n()/num_year) %>%
    select(-num_year)

# Using the proportion found above, find the mean over all years for each district.
estimation_table <- prop_data %>%
    group_by(DISTRICT) %>%
    summarise(MEAN_PROP = mean(PROP))

print("This table contains the average annual proportion of Vancouver crime in each district")
estimation_table

[1] "This table contains the average annual proportion of Vancouver crime in each district"


DISTRICT,MEAN_PROP
<chr>,<dbl>
Central,0.3710028
East,0.2888026
South,0.1748564
West,0.1719381


### Methods

#### Comment on Preliminary Results and Research Methodology 

The preliminary results only provide an overview of the mean crime proportions. Though interesting insights can be derived from the plots, we still need to advance the statistical analysis of the topic to answer the research question conclusively: 

We need a hypothesis test at a significance level of a = 0.05 to determine if there is a difference between the average annual proportion of crimes in different districts of Vancouver.

Let c = central, w = west, s = south, e = east

H0: pc = pw = ps = pe

H1: at least one {pc, pw, ps, pe} is different from the others

We will also conduct ANOVA tests to determine whether the differences between the four groups are statistically significant. So the test will analyze the levels of variance within the groups by taking samples from each of them.

### Discussion

We could not find any studies about police-reported crime rates in BC that take a micro-focused look into the districts of Vancouver and attempt to determine differences in mean crime proportion. Therefore, we believe this study could contribute to  further analysis of criminality from a community perspective. Deeper insight into crime rates across the Vancouver districts will inform citizens (as advocates for their communities) about the realities of their region, empowering them to use this information for political and community-building purposes.

### References


(STATCAN, 2021) https://www150.statcan.gc.ca/n1/pub/85-002-x/2022001/article/00013-eng.htm

(JUSTICE CANADA, 2022) https://www.justice.gc.ca/eng/rp-pr/csj-sjc/crime/rr01_1/p0.html

(GLOBE MEDIA LTD., 2022) https://www.expatarrivals.com/americas/canada/vancouver/areas-and-suburbs-vancouver

(USCIDA et al., 2014) https://www.ojp.gov/pdffiles1/nij/grants/245408.pdf